# Week 2. Descriptive Statistics and Data Cleaning

In urban data science, often the first thing we want to do is get an understanding of what the data looks like before we dive in too much. We do this by investigating the "descriptive statistics" of the data.

We might ask questions like:

What is the minimum value? What is the maximum value? Mean? Standard deviation? Is there a relationship between two variables?

Once we start looking at the data, we might find it is messy. This means there might be missing values or errors. There might be a lot of noise that we need to exclude in order to get to the signal we are looking for.

In this class, we will walk through how to explore descriptive stats for a new data set, how to identify "bad data", and how to clean your data for analysis.

In [ ]:
# Always start with your imports, we will need pandas, numpy, and matplotlib today
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

We will work with a file of census information from LA County today. Go into this week's folder on Canvas and download the file "lacounty_tracts.csv".

If you save it in the same folder as this notebook, you can load it simply as:

`pd.read_csv('lacounty_tracts.csv')`

This is called a **relative path** meaning it is relative to this notebook.

You can also always reference a file by its **absolute path**. This is the path it lives at always, irrespective to this notebook.

An absolute path looks like this:
"/Users/madilore/Documents/IUS/Week2/lacounty_tracts.csv"

You can find the path of your folder by going into your files/finder and right-clicking on the folder. You should see something that says "New Terminal at Folder". This will open up a Terminal/Command Prompt. Then, type `pwd` and hit `Enter`. This means "print working directory". This is the full file path to your folder.

Note: Windows users. Your file path might have slashes that go the other way "\\". In python, regardless of your operating system, the file path will have "/" forward slashes separating the folders.

In [ ]:
# Load the data file
df = pd.read_csv('lacounty_tracts.csv')

In [ ]:
#Let's look at what is inside the file
df.head()

The column names are:

*   `GEOID` the identifier for the census tract
*   `tot_pop`: Total population
*   `per_white`: The percent of people identifying as non-Hispanic white
*   `per_black`: The percent of people identifying as African American or non-Hispanic black
*   `per_asian`: The percent of people identifying as Asian
*   `per_hispanic`: The percent of people identifying as Hispanic
*   `med_income`: The median household income
*   `per_bach`: The percent of people with a bachelor's degree or higher
*   `per_age2534`: The percent of people aged 25-34
*   `per_age65up`: The percent of people aged 65 or older
*   `per_female`: The percent of people identifying as female
*   `h_value`: Average home value
*   `singleh`: The number of single detached units
*   `vacant`: The total number of vacant housing units
*   `hunit`: The total number of housing units
*   `dist_univ`: The distance from the center of the tract to the nearest university
*   `dist_center`: The distance from the center of the tract to the center of the city (LA)
*   `area`: The area of the census tract in sq km
*   `per_age2024`: The percent of people aged 20-24

In [ ]:
# We can start by getting descriptive stats of all the numeric variables
df.describe()

In [ ]:
# We can also see the variables plotted out to see distributions
plt.hist(df.tot_pop, bins=20) # This is a histogram with 20 bins

In [ ]:
# We can also build a histogram in pandas
df.tot_pop.plot(kind='hist', bins=20)

A powerful feature in pandas is called `groupby`. This allows you to group the data by values that are the same within a column.

In [ ]:
# For example, let's group the data by majority white (True, False)
# And then describe the percent of single detached units

df.groupby('maj_white')['singleh'].describe()

Now, recall important metrics of central tendancy:


* Mean (`df[column].mean()`): the average of all values
* Median (`df[column].median()`): the “middle” value if you order them, ie. at the 50th percentile
* Mode (`df[column].mode()`): the most common value
* Count (`df[column].count()`): the total number of values



In [ ]:
column = 'tot_pop'
print(f'The mean of {column} is {df[column].mean():.2f}')
print(f'The median of {column} is {df[column].median()}')
print(f'The mode of {column} is {df[column].mode()}')
print(f'The count of {column} is {df[column].count()}')

#What is happening with the mode?
df[column].mode()

## Data cleaning

Most of the time the data we collect is messy. We can go through a number of steps to clean it up in order to prepare the data for efficient and accurate analysis.

In [ ]:
# Let's look at our data again
df.describe()

What are some weird things we notice?

In [ ]:
df[df['med_income'] < 0]

In [ ]:
# NaN means "not a number". It represents a null or empty space
# It looks like some rows had a default of -666666666 set for missing values
# Let's set these to NaNs so we don't include them in analysis (mean, median, etc)

# We can replace it directly for all of these. Remember .loc pulls specific rows
# The rows we want are the ones that return True for:
df[df['med_income'] < 0]

df.loc[df['med_income'] < 0, 'med_income'] = np.nan

In [ ]:
# Let's look again
df.describe()

In [ ]:
# In fact, none of our variables should go below zero
# Note: only do an operation like this if you are certain you want it!
df[df < 0] = np.nan
df.describe()

In [ ]:
# It looks the counts are all over the place!
# How long is our dataframe in total?
len(df)

In [ ]:
# Look at df.describe() before and after we set the negative med_incomes to NaN
# NaNs are not included in count because they are "empty" cells
df[df['med_income'].isna()]

In [ ]:
# Sometimes only the median income is missing, but sometimes the whole row is missing
# We can see how many NaNs exist in each row
df.isna().sum(axis=1) #axis=1 means compute by row; axis=0 means by column

In [ ]:
# We can see how many of each number there is
df.isna().sum(axis=1).value_counts()

In [ ]:
# Maybe we can tolerate 1 or 2 missing/NaN values but no more than that
# Then, we want to exclude these rows

# Best practice is to make a new dataframe so you always have an original
df_clean = df[df.isna().sum(axis=1) <= 2]
df_clean.describe()

Everything is starting to look better and more clean!
It looks like our two trouble variables are `med_income` and `h_value`.

There are a few options:

1. Sometimes we are happy just leaving these as is and knowing there are some missing values. In this case, we consider the data cleaned! This might be the case when we want to get to overall patterns and metrics and small variation is okay.

In [ ]:
# This is our new dataset!
df_clean.head()

2. Othertimes, we want to exclude these variables alltogether because they are incomplete. This might be the case if we had a lot of missing values. At this point, there might be too much missing for us to consider it an accurate picture of our data.


In [ ]:
df_noincome = df.drop('med_income', axis=1) #we use axis=1 because we want to do this for every row
df_noincome.head()

3. We might also just exclude the rows that are missing these few variables. We are missing 12 rows of median income, so we might decide to exclude these rows from our analysis all together. However, we are missing 116 home values. This may be too many rows to exclude depending on what we care about. 116/2469=0.047, or 4.7% of data.

In [ ]:
df_excrows = df.loc[~df['h_value'].isna(), :]
print(len(df_excrows))
df_excrows.head()


4. Finally, we might try to estimate the missing values based on the rest of the data. We might do this if we don't have too many missing values, we think this variable is important, but estimating the value will not impact the results too much.


In [ ]:
# We might set the missing values to the average
df_newvalues = df.copy()
df_newvalues['med_income'] = df_newvalues['med_income'].fillna(df['med_income'].mean())

In [ ]:
# Or the median
df_newvalues = df.copy()
df_newvalues['med_income'] = df_newvalues['med_income'].fillna(df['med_income'].median())

In [ ]:
df_newvalues.describe()

In [ ]:
## Now let's look at some comparisons with our clean data
# Let's groupby majority white (True, False) and look at a few variables
df_clean.groupby('maj_white')['h_value'].plot(kind='hist', alpha=0.5)
plt.legend()

## Your Turn

Let's put together everything we've learned over the past two weeks with a new dataset.

Download `newyork_housing.csv` from Canvas and save it in the same folder as this notebook. Or save it in a data folder and replace the read file path below with the absolute path.

In [ ]:
ny = pd.read_csv('newyork_housing.csv')

In [ ]:
## YOUR TURN
## Invesigate the columns in the data


In [ ]:
## YOUR TURN
## Clean up any data you think it messy, an error, or missing


In [ ]:
## YOUR TURN
## Compute some descriptive stats. What might be interesting about this data?


In [ ]:
## YOUR TURN
## Plot some of the variables either alone or against one another.
## Consider grouping some of the variables first


In [ ]:
## YOUR TURN
## Write a function to compute a new variable and iterate over the values in a column

## Set it as a new column
## Pro tip: df['new_column_name'] = df['input_variable'].apply(lambda x: function_name(x))
